In [ ]:
import sys
!{sys.executable} -m pip install scikit-image tqdm torchmetrics astropy torch-summary

# Pytorch Learning Notebook  
# PCA, Classification
1 - Carbendazim  
2 - Thiacloprid  
4 - Acetamiprid   
Mixtures of the above mentioned analytes

Batch 3 of the colloids was chosen for all of the recordings due to the superior signal intensity that it showed.  

Mixtures 2 + 4 and 1 + 2 + 4 are different from all of the other data. Integration time was changed from 500ms to 1500ms due to insufficient strength of the signal

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tools.ramanflow.read_data import ReadData as rd
from tools.ramanflow.prep_data import PrepData as rpd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [ ]:
import torch

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder

## Read the data

The naming convention here as follows: NameOfAnalyte_BatchNo_ColloidsReductionSpeed_TypeOfReading  
**NameOfAnalyte**: 1 = car, 2 = thia, 4 = aceta  
**BatchNo**: Here it gets a bit tricky. The batch of colloids is the same in all of the measurements. However we splitted the acquisition into 3 separate recodrings to have a bit of variation within the batch. So the colloids that were used are all from the same batch.  
**CollidsReductionSpeed**: 3min is the time of how long it took to reduce 90ml of HH+NaOH with 10ml of AgNO3. I call it reduction speed for convinience and because it represents the matter more accurately  
**TypeOfReading**: Generally speaking there are 2 types of reading. Single spectra acquisition or spectral mapping where multiple spectra acquired. 50X50 means the mapping has 50X50 spectral images each of which corresponds to single spectra. So there are 2500 spectra in one recording.   

Total amount of data for each of the analyte and the mixtures is 7500 spectra.

#### 1 - Carbendazim

In [ ]:
f_sup, car_batch1_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1/mapping50X50/1_3min_b3_50X50_spectral_mapping_1.tif")
f_sup, car_batch2_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1/mapping50X50/1_3min_b3_50X50_spectral_mapping_2.tif")
f_sup, car_batch3_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1/mapping50X50/1_3min_b3_50X50_spectral_mapping_3.tif")

#### 2 - Thiacloprid

In [ ]:
f_sup, thia_batch1_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2/mapping 50X50/2_3min_b3_50X50_spectral_mapping_1.tif")
f_sup, thia_batch2_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2/mapping 50X50/2_3min_b3_50X50_spectral_mapping_2.tif")
f_sup, thia_batch3_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2/mapping 50X50/2_3min_b3_50X50_spectral_mapping_3.tif")

#### 4 - Acetamiprid

In [ ]:
f_sup, aceta_batch1_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 4/mapping50X50/4_3min_b3_50X50_spectral_mapping_1.tif")
f_sup, aceta_batch2_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 4/mapping50X50/4_3min_b3_50X50_spectral_mapping_2.tif")
f_sup, aceta_batch3_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 4/mapping50X50/4_3min_b3_50X50_spectral_mapping_3.tif")

### Mixtures

#### 1 + 2

In [ ]:
f_sup, car_thia_batch1_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+2/mapping50X50/1+2_3min_b3_50X50_spectral_mapping_1.tif")
f_sup, car_thia_batch2_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+2/mapping50X50/1+2_3min_b3_50X50_spectral_mapping_2.tif")
f_sup, car_thia_batch3_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+2/mapping50X50/1+2_3min_b3_50X50_spectral_mapping_3.tif")

#### 1 + 4  

**This set of measurements had some problems**  

The premixed solution of 1 and 4 that was made on 04/20 and measured on 04/21 showed strong signal.  
**However** the premixed solution of 1 and 4 that was made on 04/21 and measured on 04/22 showed no signal at all.  
In order to have a data to train on we made a decision to procede with 04/20 solution even though at the time of the experiment it was already 2 days old.  
This poses another challenge of the data reproducibility and uniformity. We normally would prefer the experimental condition to stay the same across all measurements. 

In [ ]:
f_sup, car_aceta_batch1_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+4/mapping50X50/1+4(2days)_3min_b3_50X50_spectral_mapping_1.tif")
f_sup, car_aceta_batch2_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+4/mapping50X50/1+4(2days)_3min_b3_50X50_spectral_mapping_2.tif")
f_sup, car_aceta_batch3_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+4/mapping50X50/1+4(2days)_3min_b3_50X50_spectral_mapping_3.tif")

#### 2 + 4  

As was mentioned before, this particular mixture was having a hard time producing good signal at 500ms integration time. So 1500ms was chosen instead. However the mapping size was reduced to save the time during the acquisition.  

**However, recording 6 and 7 had some issues**  

In [ ]:
f_sup, thia_aceta_batch1_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2+4/mapping 32X32/2+4_3min_b3_32X32__1500ms_spectral_mapping_1.tif")
f_sup, thia_aceta_batch2_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2+4/mapping 32X32/2+4_3min_b3_32X32__1500ms_spectral_mapping_2.tif")
f_sup, thia_aceta_batch3_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2+4/mapping 32X32/2+4_3min_b3_32X32__1500ms_spectral_mapping_3.tif")
f_sup, thia_aceta_batch4_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2+4/mapping 32X32/2+4_3min_b3_32X32__1500ms_spectral_mapping_4.tif")
f_sup, thia_aceta_batch5_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2+4/mapping 32X32/2+4_3min_b3_32X32__1500ms_spectral_mapping_5.tif")
f_sup, thia_aceta_batch6_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2+4/mapping 32X32/2+4_3min_b3_32X32_1500ms_spectral_mapping_6(problem).tif")
f_sup, thia_aceta_batch7_3min_mapping = rd.read_data("data/20220421 SERS data generation/analyte 2+4/mapping 32X32/2+4_3min_b3_32X32__1500ms_spectral_mapping_7(problem).tif")

#### 1 + 2 + 4

In [ ]:
f_sup, car_thia_aceta_batch1_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+2+4/mapping32X32/1+2+4_3min_b3_32X32_spectral_mapping_1500ms_1.tif")
f_sup, car_thia_aceta_batch2_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+2+4/mapping32X32/1+2+4_3min_b3_32X32_spectral_mapping_1500ms_2.tif")
f_sup, car_thia_aceta_batch3_3min_mapping = rd.read_data("data/20220422 SERS data generation/analyte 1+2+4/mapping32X32/1+2+4_3min_b3_32X32_spectral_mapping_1500ms_3.tif")

### MG

15000 ppb

In [ ]:
f_sup, mg_15000ppb_1 = rd.read_data("data/MG-3 15000ppb 14min/area1.tif")
f_sup, mg_15000ppb_2 = rd.read_data("data/MG-3 15000ppb 14min/area2.tif")
f_sup, mg_15000ppb_3 = rd.read_data("data/MG-3 15000ppb 14min/area3.tif")
f_sup, mg_15000ppb_4 = rd.read_data("data/MG-3 15000ppb 14min/area4.tif")

1500 ppb

In [ ]:
f_sup, mg_1500ppb_1 = rd.read_data("data/MG-4 1500ppb 0.67h/area1.tif")
f_sup, mg_1500ppb_2 = rd.read_data("data/MG-4 1500ppb 0.67h/area2.tif")
f_sup, mg_1500ppb_3 = rd.read_data("data/MG-4 1500ppb 0.67h/area3.tif")
f_sup, mg_1500ppb_4 = rd.read_data("data/MG-4 1500ppb 0.67h/area4.tif")

150 ppb

In [ ]:
f_sup, mg_150ppb_1 = rd.read_data("data/MG-5 150ppb 1.67h/area1.tif")
f_sup, mg_150ppb_2 = rd.read_data("data/MG-5 150ppb 1.67h/area2.tif")
f_sup, mg_150ppb_3 = rd.read_data("data/MG-5 150ppb 1.67h/area3.tif")
f_sup, mg_150ppb_4 = rd.read_data("data/MG-5 150ppb 1.67h/area4.tif")

## Data preprocessing

#### Collecting all three batches from each of the analyte into one matrix  

**At the same time we remove cosmic rays that may appear in individual spectra**

In [ ]:
whos

### 1

In [ ]:
car_batch1_no_cosmic_ray = np.zeros_like(car_batch1_3min_mapping)
car_batch2_no_cosmic_ray = np.zeros_like(car_batch2_3min_mapping)
car_batch3_no_cosmic_ray = np.zeros_like(car_batch3_3min_mapping)

In [ ]:
car_batch1_no_cosmic_ray = np.copy(car_batch1_3min_mapping)
car_batch2_no_cosmic_ray = np.copy(car_batch2_3min_mapping)
car_batch3_no_cosmic_ray = np.copy(car_batch3_3min_mapping)

In [ ]:
car_batch1_no_cosmic_ray = rpd.remove_cosmic_rays(car_batch1_3min_mapping, 10)

In [ ]:
car_batch2_no_cosmic_ray = rpd.remove_cosmic_rays(car_batch2_3min_mapping, 10)

In [ ]:
car_batch3_no_cosmic_ray = rpd.remove_cosmic_rays(car_batch3_3min_mapping, 10)

268, 1613, 1630, 4747, 5266, 7419

In [ ]:
list_of_fuckups = [268, 1613, 1630, 4747, 5266, 7419]

In [ ]:
car_collected_preprocessed = np.zeros((7500, 1430))
car_collected_preprocessed[0:2500] = car_batch1_3min_mapping[:, 170:]
car_collected_preprocessed[2500:5000] = car_batch2_3min_mapping[:, 170:]
car_collected_preprocessed[5000:] = car_batch3_3min_mapping[:, 170:]

In [ ]:
car_collected_preprocessed = rpd.remove_cosmic_rays(car_collected_preprocessed, 7)

In [ ]:
for i in list_of_fuckups: #len(car_collected_preprocessed)):
    plt.plot(f_sup, rpd.remove_cosmic_rays(car_collected_preprocessed[i], 7))
    # plt.plot(f_sup, car_collected_preprocessed[i])
plt.show()

### 2

In [ ]:
thia_collected_preprocessed = np.zeros((7500, 1430))
thia_collected_preprocessed[0:2500] = thia_batch1_3min_mapping[:, 170:]
thia_collected_preprocessed[2500:5000] = thia_batch2_3min_mapping[:, 170:]
thia_collected_preprocessed[5000:] = thia_batch3_3min_mapping[:, 170:]

In [ ]:
thia_collected_preprocessed = rpd.remove_cosmic_rays(thia_collected_preprocessed, 7)

In [ ]:
for i in range(len(car_collected_preprocessed)):
    plt.plot(f_sup, thia_collected_preprocessed[i])
plt.show()

### 4

In [ ]:
aceta_collected_preprocessed = np.zeros((7500, 1430))
aceta_collected_preprocessed[0:2500] = aceta_batch1_3min_mapping[:, 170:]
aceta_collected_preprocessed[2500:5000] = aceta_batch2_3min_mapping[:, 170:]
aceta_collected_preprocessed[5000:] = aceta_batch3_3min_mapping[:, 170:]

In [ ]:
aceta_collected_preprocessed = rpd.remove_cosmic_rays(aceta_collected_preprocessed, 7)

In [ ]:
for i in range(len(car_collected_preprocessed)):
    plt.plot(f_sup, aceta_collected_preprocessed[i])
plt.show()

### 1 + 2

In [ ]:
car_thia_collected_preprocessed = np.zeros((7500, 1430))
car_thia_collected_preprocessed[0:2500] = car_thia_batch1_3min_mapping[:, 170:]
car_thia_collected_preprocessed[2500:5000] = car_thia_batch2_3min_mapping[:, 170:]
car_thia_collected_preprocessed[5000:] = car_thia_batch3_3min_mapping[:, 170:]

In [ ]:
car_thia_collected_preprocessed = rpd.remove_cosmic_rays(car_thia_collected_preprocessed, 7)

In [ ]:
for i in range(len(car_collected_preprocessed)):
    plt.plot(f_sup, car_thia_collected_preprocessed[i])
plt.show()

In [ ]:
plt.plot(f_sup, np.mean(car_thia_collected_preprocessed, axis=0))

### 1 + 4

In [ ]:
car_aceta_collected_preprocessed = np.zeros((7500, 1430))
car_aceta_collected_preprocessed[0:2500] = car_aceta_batch1_3min_mapping[:, 170:]
car_aceta_collected_preprocessed[2500:5000] = car_aceta_batch2_3min_mapping[:, 170:]
car_aceta_collected_preprocessed[5000:] = car_aceta_batch3_3min_mapping[:, 170:]

In [ ]:
car_aceta_collected_preprocessed = rpd.remove_cosmic_rays(car_aceta_collected_preprocessed, 7)

In [ ]:
for i in range(len(car_collected_preprocessed)):
    plt.plot(f_sup, car_aceta_collected_preprocessed[i])
plt.show()

### 2 + 4

In [ ]:
thia_aceta_collected_preprocessed = np.zeros((7168, 1430))
thia_aceta_collected_preprocessed[0:1024] = thia_aceta_batch1_3min_mapping[:, 170:]
thia_aceta_collected_preprocessed[1024:2048] = thia_aceta_batch2_3min_mapping[:, 170:]
thia_aceta_collected_preprocessed[2048:3072] = thia_aceta_batch3_3min_mapping[:, 170:]
thia_aceta_collected_preprocessed[3072:4096] = thia_aceta_batch4_3min_mapping[:, 170:]
thia_aceta_collected_preprocessed[4096:5120] = thia_aceta_batch5_3min_mapping[:, 170:]
thia_aceta_collected_preprocessed[5120:6144] = thia_aceta_batch6_3min_mapping[:, 170:]
thia_aceta_collected_preprocessed[6144:] = thia_aceta_batch7_3min_mapping[:, 170:]

In [ ]:
thia_aceta_collected_preprocessed = rpd.remove_cosmic_rays(thia_aceta_collected_preprocessed, 7)

In [ ]:
for i in range(len(thia_aceta_collected_preprocessed)):
    plt.plot(f_sup, thia_aceta_collected_preprocessed[i])
plt.show()

### 1 + 2 + 4

In [ ]:
car_thia_aceta_collected_preprocessed = np.zeros((3072, 1430))
car_thia_aceta_collected_preprocessed[0:1024] = car_thia_aceta_batch1_3min_mapping[:, 170:]
car_thia_aceta_collected_preprocessed[1024:2048] = car_thia_aceta_batch2_3min_mapping[:, 170:]
car_thia_aceta_collected_preprocessed[2048:] = car_thia_aceta_batch3_3min_mapping[:, 170:]

In [ ]:
car_thia_aceta_collected_preprocessed = rpd.remove_cosmic_rays(car_thia_aceta_collected_preprocessed, 7)

In [ ]:
for i in range(len(car_thia_aceta_collected_preprocessed)):
    plt.plot(f_sup, car_thia_aceta_collected_preprocessed[i])
plt.show()

### MG

In [ ]:
mg_all_cntr = np.zeros((25572, 1430))
mg_all_cntr[0:2500] = mg_15000ppb_1[:, 170:]
mg_all_cntr[2500:5000] = mg_15000ppb_2[:, 170:]
mg_all_cntr[5000:7500] = mg_15000ppb_3[:, 170:]
mg_all_cntr[7500:10000] = mg_15000ppb_4[:, 170:]
mg_all_cntr[10000:12500] = mg_1500ppb_1[:, 170:]
mg_all_cntr[12500:15000] = mg_1500ppb_2[:, 170:]
mg_all_cntr[15000:17500] = mg_1500ppb_3[:, 170:]
mg_all_cntr[17500:20000] = mg_1500ppb_4[:, 170:]
mg_all_cntr[20000:22500] = mg_150ppb_1[:, 170:]
mg_all_cntr[22500:23524] = mg_150ppb_2[:, 170:]
mg_all_cntr[23524:24548] = mg_150ppb_3[:, 170:]
mg_all_cntr[24548:] = mg_150ppb_4[:, 170:]

In [ ]:
mg_all_cntr = rpd.remove_cosmic_rays(mg_all_cntr, 10)

In [ ]:
for i in range(len(mg_all_cntr)):
    plt.plot(f_sup[170:], mg_all_cntr[i])
plt.show()

### Put all of the data in one big matrix

In [ ]:
spectra_dataset = np.zeros((73312, 1430))
spectra_dataset[0:7500] = car_collected_preprocessed
spectra_dataset[7500:15000] = thia_collected_preprocessed
spectra_dataset[15000:22500] = aceta_collected_preprocessed
spectra_dataset[22500:30000] = car_thia_collected_preprocessed
spectra_dataset[30000:37500] = car_aceta_collected_preprocessed
spectra_dataset[37500:44668] = thia_aceta_collected_preprocessed
spectra_dataset[44668:47740] = car_thia_aceta_collected_preprocessed
spectra_dataset[47740:] = mg_all_cntr

In [ ]:
rpd.store_data(spectra_dataset, "clipped_dataset")

In [ ]:
spectra_dataset = rd.read_data("dataset.npy")

In [ ]:
label_mask = np.zeros((73312))

In [ ]:
label_mask[0:7500] = 1
label_mask[7500:15000] = 2
label_mask[15000:22500] = 3
label_mask[22500:30000] = 4
label_mask[30000:37500] = 5
label_mask[37500:44668] = 6
label_mask[44668:47740] = 7
label_mask[47740:] = 8

In [ ]:
label_mask[0:7500] = 1
label_mask[7500:15000] = 2
label_mask[15000:22500] = 3
label_mask[22500:30000] = (1, 2)
label_mask[30000:37500] = (1, 3)
label_mask[37500:44668] = (2, 3)
label_mask[44668:47740] = (1, 2, 3)
label_mask[47740:] = 4

In [ ]:
label_mask = np.zeros((73312, 4))

In [ ]:
label_mask[0:7500] = [1, 0, 0, 0]
label_mask[7500:15000] = [0, 1, 0, 0]
label_mask[15000:22500] = [0, 0, 1, 0]
label_mask[22500:30000] = [1, 1, 0, 0]
label_mask[30000:37500] = [1, 0, 1, 0]
label_mask[37500:44668] = [0, 1, 1, 0]
label_mask[44668:47740] = [1, 1, 1, 0]
label_mask[47740:] = [0, 0, 0, 1]

### Normalized dataset

#### Remove zeros

In [ ]:
non_zero_idx = np.where(np.sum(spectra_dataset, axis=1) != 0)
non_zero_dataset = spectra_dataset[non_zero_idx]
non_zero_label_mask = labels[non_zero_idx]

#### Remove INF or NINF

In [ ]:
non_inf_idx = np.where(np.all(np.isfinite(non_zero_dataset), axis=-1) == True)[0]
non_inf_dataset = non_zero_dataset[non_inf_idx]
non_inf_label_mask = non_zero_label_mask[non_inf_idx]

#### Use sklearn normalize

In [ ]:
norm_spectra_dataset = normalize(non_inf_dataset)

#### Split the dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(norm_spectra_dataset, non_inf_label_mask, test_size=0.2, random_state=42)

In [ ]:
y_train

### PCA on 1 analyte (Can choose any of the analytes)

In [ ]:
# car_collected_preprocessed = rpd.remove_zeros_or_nans(car_collected_preprocessed)
n_components= 7
model = PCA(n_components)
W_PCA = model.fit_transform(norm_spectra_dataset)
H_PCA = model.components_
t_dataN_PCA = H_PCA
f_dataN_PCA = W_PCA

In [ ]:
PC_values = np.arange(model.n_components_) + 1
plt.figure(figsize=(15,10))
plt.plot(PC_values, model.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

In [ ]:
np.where(non_inf_label_mask == 8)[0]

In [ ]:
plt.figure("PCA analysis on Carbendazim", figsize=(15, 10))
plt.subplot(121)#, figsize = (15, 10))
for i in range(n_components):
    plt.plot((f_dataN_PCA[:,i]-min(f_dataN_PCA[:,i]))/np.mean(max(f_dataN_PCA[:,i])-min(f_dataN_PCA[:,i]))-i)
plt.vlines(x=7500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=15000, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=22500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=30000, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=37500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=44188, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=47260, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.text(3250,1,"1")
plt.text(10000,1,"2")
plt.text(18000,1,"4")
plt.text(25000,1,"1+2")
plt.text(32000,1,"1+4")
plt.text(40000,1,"2+4")
plt.text(45000,1,"1+2+4")
plt.text(60000,1,"MG")
plt.show()
plt.subplot(122)#, figsize=(15, 10))
for i in range(n_components):
    plt.plot(((t_dataN_PCA[i]-min(t_dataN_PCA[i]))/(max(t_dataN_PCA[i])-min(t_dataN_PCA[i]))-i))
plt.show()

In [ ]:
plt.figure("PCA analysis on Carbendazim", figsize=(15, 10))
plt.subplot(121)#, figsize = (15, 10))
for i in range(n_components):
    plt.plot((f_dataN_PCA[:,i]-min(f_dataN_PCA[:,i]))/np.mean(max(f_dataN_PCA[:,i])-min(f_dataN_PCA[:,i]))-i)
# plt.vlines(x=7500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=15000, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=22500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=30000, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=37500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=44188, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=47260, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.text(3250,1,"1")
# plt.text(10000,1,"2")
# plt.text(18000,1,"4")
# plt.text(25000,1,"1+2")
# plt.text(32000,1,"1+4")
# plt.text(40000,1,"2+4")
# plt.text(45000,1,"1+2+4")
# plt.text(60000,1,"MG")
# plt.show()
plt.subplot(122)#, figsize=(15, 10))
for i in range(n_components):
    plt.plot(((t_dataN_PCA[i]-min(t_dataN_PCA[i]))/(max(t_dataN_PCA[i])-min(t_dataN_PCA[i]))-i))
plt.show()

In [ ]:
car_mean = np.mean(car_collected_preprocessed, axis=0)

In [ ]:
thia_mean = np.mean(thia_collected_preprocessed, axis=0)

In [ ]:
car_thia_mean = np.mean(car_thia_collected_preprocessed, axis=0)

In [ ]:
idx = np.where(labels == [1., 0., 0., 0.])
plt.plot(np.mean(norm_spectra_dataset[idx]))

In [ ]:
labels_np = labels.numpy()

In [ ]:
torch.where(torch.allclose(torch.eq(labels, torch.tensor([1., 1., 1., 1.]))))

In [ ]:
labels.shape

In [ ]:
pure_1_norm = np.sqrt(sum(car_mean**2))  

In [ ]:
proj_of_mix_on_1 = (np.dot(car_thia_mean, car_mean)/pure_1_norm**2)*car_mean

In [ ]:
plt.plot(proj_of_mix_on_1)
plt.plot(car_mean)

In [ ]:
spectra_dataset.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans

In [ ]:
classifier = DecisionTreeClassifier()

In [ ]:
classifier.fit(W_PCA, y_train)

In [ ]:
new_data = model.transform(X_test)

In [ ]:
pred_labels = classifier.predict(new_data)

In [ ]:
scores = cross_val_score(classifier, new_data, y_test, cv=10)

In [ ]:
print("%0.5f accuracy with a standard deviation of %0.5f" % (scores.mean(), scores.std()))

### LDA

In [ ]:
plt.figure("LDA analysis on dataset", figsize=(15, 10))
plt.subplot(121)#, figsize = (15, 10))
for i in range(7):
    plt.plot((f_dataN_LDA[:,i]-min(f_dataN_LDA[:,i]))/np.mean(max(f_dataN_LDA[:,i])-min(f_dataN_LDA[:,i]))-i)
plt.vlines(x=7500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=15000, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=22500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=30000, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=37500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=44188, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.vlines(x=47260, ymin=-5, ymax=1, colors="k", linestyles="dashed")
plt.text(3250,1,"1")
plt.text(10000,1,"2")
plt.text(18000,1,"4")
plt.text(25000,1,"1+2")
plt.text(32000,1,"1+4")
plt.text(40000,1,"2+4")
plt.text(45000,1,"1+2+4")
plt.text(60000,1,"MG")
plt.show()
plt.subplot(122)#, figsize=(15, 10))
for i in range(n_components):
    plt.plot(((t_dataN_LDA[i]-min(t_dataN_LDA[i]))/(max(t_dataN_LDA[i])-min(t_dataN_LDA[i]))-i))
plt.show()

## SVD

In [ ]:
u, s, vh = np.linalg.svd(norm_spectra_dataset, full_matrices = False)

In [ ]:
from scipy import linalg

U, s, Vh = linalg.svd(norm_spectra_dataset, full_matrices=False, lapack_driver='gesvd')

In [ ]:
U.shape

In [ ]:
s.shape

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(s[0:10])

In [ ]:
Vh.shape

In [ ]:
plt.figure("SVD analysis on dataset", figsize=(15, 10))
plt.subplot(121)#, figsize = (15, 10))
for i in range(7):
    plt.plot((Vh[i]*3) - i)
# plt.vlines(x=7500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=15000, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=22500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=30000, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=37500, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=44188, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# plt.vlines(x=47260, ymin=-5, ymax=1, colors="k", linestyles="dashed")
# # plt.text(3250,1,"1")
# plt.text(10000,1,"2")
# plt.text(18000,1,"4")
# plt.text(25000,1,"1+2")
# plt.text(32000,1,"1+4")
# plt.text(40000,1,"2+4")
# plt.text(45000,1,"1+2+4")
# plt.text(60000,1,"MG")
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
for i in range(7):
    plt.plot(U.T[i]*50 - i)
plt.show()

## Multi-label Linear Disciminant Analysis

In [ ]:
# The mean of the class

In [ ]:
# The multi-label global mean

### Class wise scatter matrices

#### Class-wise betweem-class scatter matrix

#### Class-wise within-class scatter matrix

#### Class-wise total-class scatter matrix

### One-hot label encoding

In [ ]:
labels = torch.zeros((73312, 4))

In [ ]:
labels[0:7500] = torch.tensor([1, 0, 0, 0.])
labels[7500:15000] = torch.tensor([0, 1, 0, 0])
labels[15000:22500] = torch.tensor([0, 0, 1, 0])
labels[22500:30000] = torch.tensor([1, 1, 0, 0])
labels[30000:37500] = torch.tensor([1, 0, 1, 0])
labels[37500:44668] = torch.tensor([0, 1, 1, 0])
labels[44668:47740] = torch.tensor([1, 1, 1, 0])
labels[47740:] = torch.tensor([0, 0, 0, 1])

In [ ]:
non_zero_idx = np.where(np.sum(spectra_dataset, axis=1) != 0)
labels = labels[non_zero_idx]

In [ ]:
non_inf_idx = np.where(np.all(np.isfinite(non_zero_dataset), axis=-1) == True)[0]
labels = labels[non_inf_idx]

In [ ]:
labels.shape

In [ ]:
from spectral_dataset import *

In [ ]:
train_load, val_load, test_load = spectral_dataloaders(norm_spectra_dataset, labels)

In [ ]:
from torchvision import models
from torch import nn

In [ ]:
from torchvision import datasets, models, transforms
import torch.nn as nn

In [ ]:
n_input = non_inf_dataset.shape[-1]

In [ ]:
model = nn.Sequential(nn.Conv1D(n_input, n_hidden),
                      nn.ReLU(),
                      nn.Conv1D(),
                      nn.ReLU(),
                      nn.MaxPool1d(),
                      nn.Dropout(),
                      nn.Conv1D(),
                      nn.Conv1D(),
                      nn.MaxPool1d(),
                      nn.Dropout(),
                      nn.Conv1D(),
                      nn.Conv1D(),
                      nn.MaxPool1d(),
                      nn.Flatten(),
                      nn.Linear(),
                      nn.Linear(n_hidden, n_out),
                      nn.Sigmoid())
print(model)

### For labels encoding use Multi Hot Encoding

### For accuracy metrics use F1 Score

In [ ]:
from torchmetrics import F1Score

### For Loss function use Binary Cross Entropy with Logits  
This loss tends to perform best for multilabel classification

In [ ]:
from resnet import ResNet
import os

In [ ]:
# CNN parameters
layers = 6
hidden_size = 50
block_size = 2
hidden_sizes = [hidden_size] * layers
num_blocks = [block_size] * layers
input_dim = 1430
in_channels = 64
n_classes = 4
os.environ['CUDA_VISIBLE_DEVICES'] = '{}'.format(0)
cuda = torch.cuda.is_available()

In [ ]:
# Load trained weights for demo
cnn = ResNet(hidden_sizes, num_blocks, input_dim=input_dim,
                in_channels=in_channels, n_classes=n_classes)

# if cuda: 
cnn.cuda()

In [ ]:
from training import run_epoch
from torch import optim
from time import time

In [ ]:
X_train.shape

In [ ]:
p_val = 0.1
n_val = int(58224 * p_val)
idx_tr = list(range(58224))
np.random.shuffle(idx_tr)
idx_val = idx_tr[:n_val]
idx_tr = idx_tr[n_val:]

In [ ]:
# Fine-tune CNN
epochs = 1 # Change this number to ~30 for full training
batch_size = 10
t0 = time()
# Set up Adam optimizer
optimizer = optim.Adam(cnn.parameters(), lr=1e-3, betas=(0.5, 0.999))
# Set up dataloaders
dl_tr = spectral_dataloader(X_train, y_train, idxs=idx_tr,
    batch_size=batch_size, shuffle=True)
dl_val = spectral_dataloader(X_train, y_train, idxs=idx_val,
    batch_size=batch_size, shuffle=False)
dl_test = spectral_dataloader(X_test, y_test, batch_size=10, shuffle=False)
# Fine-tune CNN for first fold
best_val = 0
no_improvement = 0
max_no_improvement = 5
print('Starting fine-tuning!')
for epoch in range(epochs):
    print(' Epoch {}: {:0.5f}s'.format(epoch+1, time()-t0))
    # Train
    acc_tr, loss_tr, acccc_tr, lossss_tr = run_epoch(epoch, cnn, dl_tr, cuda,
        training=True, optimizer=optimizer)
    print('  Train acc: {:0.5f}'.format(acc_tr))
    # Val
    acc_val, loss_val, acccc_val, lossss_val = run_epoch(epoch, cnn, dl_val, cuda,
        training=False, optimizer=optimizer)
    print('  Val acc  : {:0.5f}'.format(acc_val))
    # Check performance for early stopping
    if acc_val > best_val or epoch == 0:
        
        best_val = acc_val
        no_improvement = 0
    else:
        no_improvement += 1
    if no_improvement >= max_no_improvement:
        print('Finished after {} epochs!'.format(epoch+1))
        break

print('\n This demo was completed in: {:0.2f}s'.format(time()-t0))

In [ ]:
from torchsummary import summary
summary(cnn)

In [ ]:
len(acccc_tr)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(acccc_tr)

In [ ]:
lossss_tr[-1]

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(lossss_tr)

## Making Predicitions

In [ ]:
from training import get_predictions
# from spectral_datasets import spectral_dataloader
from sklearn.metrics import f1_score

In [ ]:
# Make predictions on subset of data
t0 = time()
# dl = spectral_dataloader(X, y, batch_size=10, shuffle=False)
y_hat = get_predictions(cnn, dl_test, cuda, get_probs = True)
print('Predicted {} spectra: {:0.5f}s'.format(len(y_hat), time()-t0))

In [ ]:
y_test.detach().cpu().numpy()

In [ ]:
# Computing accuracy
# acc = (y_hat == y).mean()
acc = f1_score((y_hat > 0.6).astype(int), y_test.detach().cpu().numpy().astype(int), average='macro')
print('Accuracy: {:0.5f}%'.format(100*acc))

#### Make predictions on synthetic data
The result should go to last label.  
This data was not used during training.  

In [ ]:
def noise_aug(y_data: np.ndarray, amp: float):
    """
    Adds gaussian noise to the spectrum in direction of its variance.

    Parameters
    ----------
    y_data : np.ndarray
        Y data. A spectrum.
    amp : float
        A percentage given as a decimal. Strength of noise.

    Returns
    -------
    np.ndarray
        Spectrum plus noise.

    """    
    #adding noise to the spectrum
    n = np.random.normal(0, y_data.std(), y_data.shape[-1]) * amp
    return y_data + n

In [ ]:
X_test_noisy = noise_aug(X_test, amp=0.3)

In [ ]:
plt.plot(X_test[45])

In [ ]:
plt.plot(X_test_noisy[45])

In [ ]:
dl_test_noisy = spectral_dataloader(X_test_noisy, y_test, batch_size=10, shuffle=False)

In [ ]:
# Make predictions on subset of data
t0 = time()
# dl = spectral_dataloader(X, y, batch_size=10, shuffle=False)
y_hat = get_predictions(cnn, dl_test_noisy, cuda, get_probs = True)
print('Predicted {} spectra: {:0.5f}s'.format(len(y_hat), time()-t0))

In [ ]:
y_test.detach().cpu().numpy()
# Computing accuracy
# acc = (y_hat == y).mean()
acc = f1_score((y_hat > 0.6).astype(int), y_test.detach().cpu().numpy().astype(int), average='macro')
print('Accuracy: {:0.5f}%'.format(100*acc))

In [ ]:
from SyntheticSignal import Synthetic_Signal as synsi

In [ ]:
def create_dataset(params, add_peak, shuffle=False):
    dat = list()
    piks = list()
    for i in range(len(params['smu'])):
        siegs = synsi.Synthetic_Signal()
        siegs.generate_random_walk(N=params['N'], batch_size=params['batch_size'],
                                   mu=params['smu'][i],sigma=params['ssig'][i],
                                   reverse=params['reverse'][i], seed=params['sid'][i])
        siegs.smooth_data()
        if(add_peak[i]):
            siegs.r_add_peak(fwhmG=params['fwhmG'][i], fwhmL=params['fwhmL'][i],
                             amplitudeL=params['pa'][i])
            piks.append([siegs.peak[i].x_0[0] for i in range(siegs.batch_size)])
        else:
            piks.append([0 for i in range(siegs.batch_size)])
        siegs.add_noise(mu=params['nmu'][i], sigma=params['nsig'][i],
                        a=params['na'][i])
        dat.append(siegs.data)
    peaks = np.concatenate((piks[0], piks[1], piks[2], piks[3]), axis=0)
    dataset = np.concatenate((dat[0], dat[1], dat[2], dat[3]), axis=0)
    #if(shuffle):
        
    return dataset, peaks

In [ ]:
params = {'N':1430, 'batch_size':30, 'smu':[0, 0, 1, 2], 'ssig':[1, 2, 0.5, 1],
          'reverse':[True, False, True, True], 'sid':[21, 13, 15, 24],
          'fwhmG':[53, 25, 32, 45], 'fwhmL':[15, 10, 5, 21], 'pa':[0.9, 0.6, 0.5, 1],
          'nmu':[0, 0, 0, 0], 'nsig':[1, 1, 1, 1], 'na':[0.03, 0.06, 0.09, 0.12]}

add_peak = [True, True, False, True]

shuffle = False

In [ ]:
dataset, peak_locs = create_dataset(params, add_peak, shuffle)

In [ ]:
dataset.shape

In [ ]:
plt.plot(dataset[100])

In [ ]:
labels_synthetic = torch.zeros((120, 4))
labels_synthetic[:] = torch.tensor([0, 0, 0, 1])

In [ ]:
labels_synthetic.shape

In [ ]:
dl_test_synthetic = spectral_dataloader(dataset, labels_synthetic, batch_size=10, shuffle=False)

In [ ]:
# Make predictions on subset of data
t0 = time()
# dl = spectral_dataloader(X, y, batch_size=10, shuffle=False)
y_hat = get_predictions(cnn, dl_test_synthetic, cuda, get_probs = True)
print('Predicted {} spectra: {:0.5f}s'.format(len(y_hat), time()-t0))

In [ ]:
# labels_synthetic.detach().cpu().numpy()
# Computing accuracy
# acc = (y_hat == y).mean()
acc = f1_score((y_hat > 0.6).astype(int), labels_synthetic.detach().cpu().numpy().astype(int), average='macro')
print('Accuracy: {:0.5f}%'.format(100*acc))